# 코드로 인해 구현해보기

In [40]:
# KoBERT fine tuning 할 수 있는 버전 가져오기
# Github 업로드 시 줄이 불필요하게 길어질 부분 주석 처리

# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [2]:
# KoBERT Fine-Tuning에 필요한 패키지 import

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm


# kobert
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

from transformers import AdamW # 모델의 초기값 지정 함수를 아담으로 설정.
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd

In [3]:
# Colab 환경에서 진행하기 때문에 google drive 연결

from google.colab import drive

drive.mount('/content/drive')

# GPU 사용.
device = torch.device("cuda:0") 
# device = torch.device("cpu")

Mounted at /content/drive


In [4]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [5]:
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.10.1+cu102
cuda version: 10.2
cudnn version:7605


### KoBert를 활용한 네이버 뉴스 분류기
Sector : 카테고리별 상이
- 정치 : 100
- 경제 : 101
- 사회 : 102
- 생활/문화 : 103
- 세계 : 104
- IT/과학 : 105
- 연애 : 106
- 스포츠 : 107

In [6]:
data = pd.read_csv('/content/drive/MyDrive/final_df_news.csv',encoding='utf-8-sig',index_col=0)

In [7]:
data.head()

,sector,date,page,news_content
0,100,20230127,1,올해 첫번째 담화 발표러시아와 연대 재확인우크라 전황에 위기감?김정은 북한 국무위원...
0,100,20230127,1,이종섭 국방부 장관과 로이드 오스틴 미국 국방장관이 지난해 7월 29일(현지시간...
0,100,20230127,1,제프 자이언츠 신임 비서실장[로이터 연합뉴스 자료사진. 재판매 및 DB 금지](워싱...
0,100,20230127,1,김여정 노동당 중앙위 부부장 / 사진 = 연합뉴스북한이 미국의 우크라이나 탱크 지원...
0,100,20230127,1,"러시아 지지하며 ""미국과 서방의 무기, 러 앞에 파철더미 될 것""김여정 북한 노동당..."


In [8]:
data = data[['news_content','sector']]

In [9]:
data.reset_index(drop=True,inplace=True)
data.drop_duplicates(subset=['news_content'],inplace=True)
data.dropna(axis=0,inplace=True)

In [10]:
def labeling_sector(data):
  if data == 100:
    return '0'
  elif data == 101:
    return '1'
  elif data == 102:
    return '2'
  elif data == 103:
    return '3'
  elif data == 104:
    return '4'
  elif data == 105:
    return '5'

In [11]:
data['label_sector'] = data['sector'].apply(lambda x: labeling_sector(x))

In [12]:
# def translate(data):
#   return data.translate(str.maketrans('','',string.punctuation))

In [13]:
# from tqdm import tqdm
# for idx,text in tqdm(enumerate(data)):
#   if type(text[0]) == float:
#     print(text[0])

In [14]:
data = data[['news_content','label_sector']]
# data = [[i, str(j)] for i, j in zip(data['news_content'], data['label_sector'])]
data = [[i,j,] for i,j in zip(data['news_content'],data['label_sector'])]
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [15]:
# 토큰화

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int64(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
## Setting parameters

max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data, test_size = 0.2, random_state=0)


data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=1)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=1)

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [21]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [22]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [23]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [24]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [25]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [26]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [27]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/3696 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7756121158599854 train acc 0.265625
epoch 1 batch id 201 loss 1.5550557374954224 train acc 0.24191542288557213
epoch 1 batch id 401 loss 0.8750314116477966 train acc 0.45343672069825436
epoch 1 batch id 601 loss 0.6706817746162415 train acc 0.5572223377703827
epoch 1 batch id 801 loss 0.6883760690689087 train acc 0.6119499063670412
epoch 1 batch id 1001 loss 0.7149335741996765 train acc 0.6454483016983017
epoch 1 batch id 1201 loss 0.5114542245864868 train acc 0.6684273522064946
epoch 1 batch id 1401 loss 0.5700802803039551 train acc 0.6854701998572448
epoch 1 batch id 1601 loss 0.7104976773262024 train acc 0.6991919113054341
epoch 1 batch id 1801 loss 0.5065160393714905 train acc 0.7090765546918378
epoch 1 batch id 2001 loss 0.4559873938560486 train acc 0.7179691404297851
epoch 1 batch id 2201 loss 0.4257538914680481 train acc 0.7253805088596093
epoch 1 batch id 2401 loss 0.5933232307434082 train acc 0.731472563515202
epoch 1 batch id 2601 loss 0.284853696823

  0%|          | 0/924 [00:00<?, ?it/s]

epoch 1 test acc 0.8126076101928376


  0%|          | 0/3696 [00:00<?, ?it/s]

...

epoch 4 test acc 0.84227358815427


  0%|          | 0/3696 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.04535792022943497 train acc 0.984375
epoch 5 batch id 201 loss 0.08693170547485352 train acc 0.9386660447761194
epoch 5 batch id 401 loss 0.21229740977287292 train acc 0.9371493142144638
epoch 5 batch id 601 loss 0.15863381326198578 train acc 0.9383059484193012
epoch 5 batch id 801 loss 0.2624621093273163 train acc 0.938455836454432
epoch 5 batch id 1001 loss 0.41754260659217834 train acc 0.9393106893106893
epoch 5 batch id 1201 loss 0.2117055207490921 train acc 0.9399198584512906
epoch 5 batch id 1401 loss 0.16474926471710205 train acc 0.9406673804425411
epoch 5 batch id 1601 loss 0.30012091994285583 train acc 0.9413745315427857
epoch 5 batch id 1801 loss 0.10371897369623184 train acc 0.9422543031649084
epoch 5 batch id 2001 loss 0.05728113651275635 train acc 0.9429269740129935
epoch 5 batch id 2201 loss 0.06822067499160767 train acc 0.9435270899591095
epoch 5 batch id 2401 loss 0.19971132278442383 train acc 0.9440337359433569
epoch 5 batch id 2601 loss 0.024

  0%|          | 0/924 [00:00<?, ?it/s]

epoch 5 test acc 0.8443676825068871


In [28]:
torch.save(model, f'/content/drive/MyDrive/News_classification_KOBert.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/News_classification_KOBert_StateDict.pt')

In [31]:
import pandas as pd
from tqdm import tqdm, tqdm_notebook

In [34]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

def testModel(model, seq):
    cate = ["정치","경제","사회","생활/문화","세계","IT/과학","연애","스포츠"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    model.eval()
    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    return cate[idx]

In [35]:
# 정치 기사 예시

ex_sentence = '''(서울=뉴스1) 허고운 기자 = 미군 당국이 22일 진행된 한미일 3국의 미사일 방어훈련 장소를 동해(East Sea)가 아닌 일본해(Sea of Japan)로 표기한 것과 관련, 우리 측이 수정을 요청한 것으로 확인됐다.
이성준 합동참모본부 공보실장은 23일 정례브리핑에서 "미 인도·태평양사령부는 (훈련 장소를) '일본해'라고 표기했고 아직 변경하지 않은 상태"라며 "한국은 미국 측에 그런 사실을 수정해 줄 것을 요구했다"고 밝혔다.

이 실장은 "한국의 입장을 미국 측에 전달한 만큼 그 결과를 지켜볼 것"이라며 "각국의 서로 다른 입장을 고려한 결과가 나오지 않을까 생각한다"고 말했다.

전날 동해 울릉도 동쪽 공해상에선 우리 해군 구축함 '세종대왕함'과 미 해군 구축함 '배리', 그리고 일본 해상자위대 호위함 '아타고' 등 3척의 이지스함이 참여한 미사일 방어훈련이 실시됐다. 훈련 해역은 독도에서 동쪽으로 약 185㎞, 일본 본토에선 서쪽으로 약 120㎞ 떨어진 곳이었던 것으로 알려졌다.

이와 관련 미 인도·태평양사령부는 이번 훈련 사실을 공표하면서 그 장소를 '일본해'로 표기해 논란이 일고 있다.'''

In [36]:
testModel(model, ex_sentence)

'정치'

In [39]:
# 경제 기사 예시

ex_sentence_1 = '''정부가 난방비 지원을 차질 없이 신속하게 집행할 수 있도록 전담 조직을 운영합니다. 산업통상자원부는 "난방 공급자에 따라 지원방식, 지원 시기 등이 차이가 있어 현장의 혼선을 방지하고, 관련 기관 간 협조체계 마련을 위해 구성됐다"고 밝혔습니다.
도시가스 이용자 가운데 기존 지원 대상자는 별도의 신청이 필요 없습니다. 신규 신청자는 관할 지역 주민센터나 도시가스사로 신청하면 지원액을 차감한 금액이 청구됩니다.
한국지역난방공사 이용자는 기존대상자여도 새로 신청을 해야 합니다. 지난해 12월부터 올해 3월까지의 청구서를 첨부해 오는 4월과 5월 중으로 한국지역난방공사로 난방비 지원을 신청하면 대상자와 지원금액을 검증한 뒤 계좌로 현금이 지급될 예정입니다.
지원에 대한 자세한 사항은 지역난방공사 고객상담센터(☏1688-2488)로 문의하면 됩니다. 집단에너지협회는 지역난방 민간사업자 이용자를 지원할 계획이며 접수처는 아직 정하지 못했습니다.
지난 1월과 2월 이용 금액에 대해 최대 59만 2천 원을 지원하는 방안을 현재 검토하고 있습니다.'''

In [38]:
testModel(model, ex_sentence_1)

'경제'